# Data Cleaner

Different scripts to clean and trim our data

**TODO: make rs values integers by removing "rs"**

In [2]:
from db_scripts import *

GWAS dataset:

In [13]:
fileIn = getPath('gwas_catalog_v1.0-associations_e108_r2023-01-14.tsv') # https://www.ebi.ac.uk/gwas/docs/file-downloads
# fileOut = getPath('gwas_trimmed_beeg.tsv')
fileOut = getPath('gwas_trimmed.tsv')

data = pd.read_csv(fileIn, sep='\t', low_memory=False)    # Reads gwas tsv
data=removeSpecial(data)    # removes special characters in column names
data=data.query("DISEASE_TRAIT=='Type 1 diabetes' or STUDY.str.contains('type 1 diabetes')")
data = data.loc[data.SNPS.str.contains(r'rs[0-9]+')]        # get only snps with rsids
#data = data.loc[data['CHR_ID']=='6']                        # Select only rows for chromosome 6

data = data[["SNPS","REGION","CHR_POS","P_VALUE","MAPPED_GENE"]] # maybe also include STRONGEST SNP-RISK ALLELE and RISK ALLELE FREQUENCY ?

data=removeDupeSNP(data)    # Remove duplicates (leaving the entry with largest p value)
data.rename(columns = {'SNPS':'rsid'}, inplace = True)

# if os.path.exists(fileOut): # If the file exists,
#     os.remove(fileOut)     # delete it.
data.to_csv(fileOut, sep='\t')

Functional data:

In [14]:
# fileIn=getPath('Functional_and_Ontology_data.tsv')
fileIn=getPath('func_data.tsv')
fileOut=getPath('Func_trimmed.tsv')

data=pd.read_csv(fileIn,sep='\t')
dupeList = data.duplicated(subset=['#Uploaded_variation','Allele'])   # Get list of duplicate values
data=data[~dupeList]
data=removeSpecial(data)
data.rename(columns = {'Uploaded_variation':'rsid'}, inplace = True)
data = data[['rsid','Allele','CADD_PHRED', 'CADD_RAW']]#, 'PolyPhen', 'SIFT']] # maybe re-add polyphen and sift later

# data=castRS(data, "Uploaded_variation") # converts rs value column to integer
data.to_csv(fileOut, sep='\t')

Population data:

In [19]:
fileIn=getPath('population_variation.tsv')
fileOut=getPath('population_variation_noSpecial.tsv')

data=pd.read_csv(fileIn,sep='\t')
data=removeSpecial(data)        # Removes special characters
data.rename(columns = {'SNP_rsID':'rsid'}, inplace = True)

# data=castRS(data,"SNP_rsID")    # converts rs value column to integer
data.to_csv(fileOut, sep='\t')

Func data 2:

In [12]:
if 0:   # disable for the time being
    fileIn=getPath('Functional_and_Ontology_data.tsv')
    fileOut=getPath('Func_trimmed.tsv')

    Association_table_filename = 'Functional_and_Ontology_data.tsv'
    df = pd.read_csv(fileIn, sep='\t')
    columns_to_keep = ['#Uploaded_variation','Allele','CADD_PHRED', 'CADD_RAW', 'PolyPhen', 'SIFT']
    df = df[columns_to_keep]
    # print(df)
    df=removeSpecial(df)    # removes special characters in column names
    df.to_csv(fileOut,sep='\t')